In [1]:
import os
import pandas as pd
import pickle

from dotenv import load_dotenv
from routingGA import RoutingGA
from domain.vehicle import Vehicle
from domain.stop import Stop
from abi.libs.rust_solver_lib import GAParameters
from abi.libs.cpp_solver_lib import GALib
from abi.libs.rust_solver_lib import RustSolverLib
from data.stops import stops
from distancesAPI import MapsAPI

%reload_ext autoreload
%autoreload 2

load_dotenv(override=True)

True

In [2]:
refetch_distances = False
calculate_time = False
plot_markers = False

In [3]:
addresses = [x.address for x in stops]
mapsAPI = MapsAPI(os.getenv('API_KEY'))
address_chunks = MapsAPI.split_in_chunks(addresses, 25)
distances = mapsAPI.split_distance_request(address_chunks, calculateTime = calculate_time) if refetch_distances else MapsAPI.get_from_cache()
distances = MapsAPI.convert_to_symmetric(distances)

In [4]:
df = pd.DataFrame(distances)

In [5]:
with open('cache/distance_matrix', 'rb') as f:
    best_ever = pickle.load(f)

In [6]:
def insert_ids_in_distances(distances):
    return [[src_id, dest_id, distance] for src_id, sublist in enumerate(distances) for dest_id, distance in enumerate(sublist)]


In [53]:
distances_with_ids = insert_ids_in_distances(distances)
vehicles = [Vehicle(0, 1000), Vehicle(1, 1000), Vehicle(2, 1000), Vehicle(3, 1000)]
parameters = GAParameters(population_size=200, elite_size=8, mutation_rate=0.05, local_search_rate=0.2, max_crossover_tries=10, max_generations=1000)
rust_lib = RustSolverLib(os.getenv('RUST_SOLVER_LIB'))
rust_result = rust_lib.run_genetic_solver(vehicles[:2], stops, distances_with_ids, parameters)
print(sum(x.total_distance for x in rust_result))
[print([y.id for y in x.stops]) for x in rust_result]

8672.0
[0, 10, 8, 6, 7, 5, 3, 4, 31, 1, 15, 24, 19, 21, 13, 14, 0]
[0, 12, 11, 16, 25, 26, 23, 22, 18, 17, 27, 9, 20, 32, 30, 29, 28, 2, 0]


[None, None]

In [8]:
routingGA = RoutingGA(popSize=200, qtyLocations=len(distances) - 1, qtyRoutes=3,
                          maxGenerations=800, selectionK=5, mutationRate=0.05, optRate=0.02,distances=distances)
                           
lib = GALib(routingGA=routingGA,
            libPath=os.getenv('CPP_SOLVER_LIB'))

In [9]:
result = lib.run()
print(routingGA.calculate_total_distance(result))
print([len(x) for x in result])
result

9872
[8, 11, 1, 25]


[[0, 30, 3, 32, 9, 10, 6, 20],
 [0, 15, 13, 27, 14, 12, 1, 4, 11, 22, 23],
 [0],
 [0,
  13,
  27,
  14,
  12,
  1,
  4,
  11,
  22,
  23,
  21,
  17,
  26,
  19,
  2,
  5,
  29,
  18,
  8,
  25,
  28,
  7,
  24,
  16,
  31]]

In [10]:
best_n2 = [[0, 15, 26, 9, 23, 19, 28, 31, 21, 2, 25, 32, 22, 17, 27, 18, 10, 14],
 [0, 11, 6, 8, 5, 7, 16, 12, 4, 20, 3, 30, 29, 24, 13, 1]]

In [11]:
best_n3 = [[0, 29, 31, 19, 28, 15, 23, 30, 24, 22, 18],
 [0, 3, 16, 21, 14, 13, 10, 27, 25, 6, 7, 9, 17, 8, 20, 2, 11, 1, 12],
 [0, 5, 26, 4, 32]]

In [12]:
best_n4 = [[0, 30, 29, 31],
 [0, 11, 15, 12, 14, 5, 22, 1, 13],
 [0, 4, 3, 28, 2, 20, 19, 17, 27, 25, 24, 10, 8, 7, 9, 6, 32, 26, 16, 21, 18, 23]]

In [13]:
print(routingGA.calculate_total_distance(result))
print(sum(x.total_distance for x in rust_result))

9872
7761.0


In [14]:
# with open('cache/best_ever', 'wb'):
#     pickle.dumps(best)

In [15]:
# route_addresses = [[addresses[y] for y in x] for x in best]
# results = mapsAPI.split_directions_request(route_addresses[0])
# mapsAPI.save_to_image(results, f"route_test")

In [16]:
# route_addresses = [[addresses[y] for y in x] for x in best_n4]
# imageCount = 1
# for route in route_addresses:
#     directions = mapsAPI.directions(route[0], route[0], route[1: 25])
#     mapsAPI.save_to_image(directions, f"n4_route_{imageCount}")
#     imageCount += 1

In [17]:
if plot_markers:
    plot = mapsAPI.plot_markers(addresses)
    with open('test.jpg', 'wb') as f:
        for chunk in plot:
            f.write(chunk)